In [3]:
# Import libraries
import numpy as np
import pandas as pd

In [4]:
# URL from STL FRED with all daily prices
historical_price_url = 'https://fred.stlouisfed.org/data/DCOILWTICO.txt'

In [5]:
raw_data = pd.read_csv(historical_price_url)

In [6]:
# How many rows should we drop to get to data
raw_data.head(15)

,Title: Crude Oil Prices: West Texas Intermediate (WTI) - Cushing,Oklahoma
0,Series ID: DCOILWTICO,NaN
1,Source: U.S. Energy Information A...,NaN
2,Release: Spot Prices (Not a Press ...,NaN
3,Seasonal Adjustment: Not Seasonally Adjusted,NaN
4,Frequency: Daily,NaN
5,Units: Dollars per Barrel,NaN
6,Date Range: 1986-01-02 to 2022-05-16,NaN
7,Last Updated: 2022-05-18 12:11 PM CDT,NaN
8,Notes: Definitions,Sources and Explanatory Notes
9,(http://www.eia.doe.gov/d...,NaN


In [7]:
# Drop initial rows of non-price information
df = raw_data.tail(-10)
df

,Title: Crude Oil Prices: West Texas Intermediate (WTI) - Cushing,Oklahoma
10,DATE VALUE,NaN
11,1986-01-02 25.56,NaN
12,1986-01-03 26.00,NaN
13,1986-01-06 26.53,NaN
14,1986-01-07 25.85,NaN
...,...,...
9494,2022-05-10 99.74,NaN
9495,2022-05-11 105.50,NaN
9496,2022-05-12 106.15,NaN
9497,2022-05-13 110.52,NaN


In [8]:
# Rename columns, drop first row, and drop 'OK' column
df.columns = ['Date Value','OK']
prices_df = df.iloc[1:].drop(columns='OK')

prices_df

,Date Value
11,1986-01-02 25.56
12,1986-01-03 26.00
13,1986-01-06 26.53
14,1986-01-07 25.85
15,1986-01-08 25.87
...,...
9494,2022-05-10 99.74
9495,2022-05-11 105.50
9496,2022-05-12 106.15
9497,2022-05-13 110.52


### We are going to need to break up the 'Date Value' column into 2 columns 
We will then convert the Value column into a float.
Then we will convert the date column to datetime and make it the index

In [9]:
# We need to convert df['Value'] to a float but we have these almost empty strings '   .' instead of nulls so we can't use .astype(float) on the column
# We need to figure out which error it creates so we can make a try-except statement
float('       .')

ValueError: could not convert string to float: '       .'

In [10]:
# Create a function that will convert the 'Value' to a float if it has a numerical equivalent or return Null if it is a string
def float_or_null(x):
    try:
        return float(x)
    except ValueError:
        return np.nan


In [11]:
# Test our function to make sure it works
print(float_or_null('3.65'))
print(float_or_null('       .'))

3.65
nan


In [12]:
# Split the column into 2 columns
prices_df['Date'] = prices_df['Date Value'].str.slice(0,10)
prices_df['Value'] = prices_df['Date Value'].str.slice(10,).apply(float_or_null)

In [13]:
prices_df

,Date Value,Date,Value
11,1986-01-02 25.56,1986-01-02,25.56
12,1986-01-03 26.00,1986-01-03,26.00
13,1986-01-06 26.53,1986-01-06,26.53
14,1986-01-07 25.85,1986-01-07,25.85
15,1986-01-08 25.87,1986-01-08,25.87
...,...,...,...
9494,2022-05-10 99.74,2022-05-10,99.74
9495,2022-05-11 105.50,2022-05-11,105.50
9496,2022-05-12 106.15,2022-05-12,106.15
9497,2022-05-13 110.52,2022-05-13,110.52


In [14]:
# Test to see that Nulls were created and how many
prices_df['Value'].isnull().sum()

323

In [15]:
# Confirm that ['Values'] is now float
prices_df.dtypes

Date Value     object
Date           object
Value         float64
dtype: object

In [19]:
# Now we need to drop the original 'Date Value' column, convert the 'Date' to datetime and set as index
prices_df.columns = ['Drop','Date','$/BBL']
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
historical_prices_df = prices_df.drop(columns=['Drop']).set_index(['Date']).sort_index()

historical_prices_df

,$/BBL
Date,
1986-01-02,25.56
1986-01-03,26.00
1986-01-06,26.53
1986-01-07,25.85
1986-01-08,25.87
...,...
2022-05-10,99.74
2022-05-11,105.50
2022-05-12,106.15


In [20]:
# Roll it up to get monthly averages
monthly_historical_prices = historical_prices_df.resample(rule='MS').mean().round(2)

In [21]:
monthly_historical_prices

,$/BBL
Date,
1986-01-01,22.93
1986-02-01,15.45
1986-03-01,12.61
1986-04-01,12.84
1986-05-01,15.38
...,...
2022-01-01,83.22
2022-02-01,91.64
2022-03-01,108.50


### Now we need to pull future prices
Scrape this from *CME Group website*

In [22]:
# Import Splinter, datetime, and time
from splinter import Browser
from datetime import datetime
import time

In [25]:
# Path to chromedriver
!which chromedriver

/usr/local/bin/chromedriver


In [26]:
# Set the executable path and initialize the chrome browser in splinter
# Is chromedriver up to date?
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path) #, headless=True)

In [27]:
# Navigate to website with oil futures
wti_site = 'https://www.cmegroup.com/markets/energy/crude-oil/light-sweet-crude.quotes.html'
browser.visit(wti_site)

In [40]:
# Wait 6 seconds before clicking
#time.sleep(6)

# # Accept cookies so we can load all data
# print(browser.links.find_by_partial_text("Accept All"))#.click()

# Manually accept until we can fix this

''

In [41]:
# ElementClick was intercepted so need to wait before clicking "LOAD ALL"


# Load all by clicking button
element = browser.find_by_css("button.primary.load-all")

browser.execute_script("window.scrollTo(0,2000);")
time.sleep(2)
element.click()

In [42]:
# Read html data into pandas
html = browser.html
wti_prices = pd.read_html(html)

In [45]:
# List of dataframes, view the first (and only) one
wti_prices[0]

,Month,Options,Chart,Last,Change,PriorSettle,Open,High,Low,Volume,Updated
,Month,Options,Chart,Last,Change,PriorSettle,Open,High,Low,Volume,Updated
0,JUL 2022CLN2,Opt,NaN,110.84,+0.56 (+0.51%),110.28,110.56,111.20,109.51,17492,23:09:27 CT 22 May 2022
1,AUG 2022CLQ2,Opt,NaN,107.75,+0.58 (+0.54%),107.17,107.34,108.03,106.47,2940,23:09:14 CT 22 May 2022
2,SEP 2022CLU2,Opt,NaN,104.55,+0.61 (+0.59%),103.94,104.11,104.75,103.39,1122,23:09:14 CT 22 May 2022
3,OCT 2022CLV2,Opt,NaN,101.69,+0.62 (+0.61%),101.07,100.85,101.78,100.56,184,22:44:28 CT 22 May 2022
4,NOV 2022CLX2,Opt,NaN,99.39,+0.69 (+0.70%),98.70,98.31,99.39,98.09,168,22:14:21 CT 22 May 2022
...,...,...,...,...,...,...,...,...,...,...,...
123,OCT 2032CLV32,Opt,NaN,-,-,64.58,-,-,-,0,17:02:03 CT 22 May 2022
124,NOV 2032CLX32,Opt,NaN,-,-,64.67,-,-,-,0,17:02:03 CT 22 May 2022
125,DEC 2032CLZ32,Opt,NaN,-,-,64.62,-,-,-,0,17:02:03 CT 22 May 2022


In [44]:
browser.quit()

In [96]:
# Save scraped prices to a dataframe
wti_prices_df = wti_prices[0]
wti_prices_df

,Month,Options,Chart,Last,Change,PriorSettle,Open,High,Low,Volume,Updated
,Month,Options,Chart,Last,Change,PriorSettle,Open,High,Low,Volume,Updated
0,JUL 2022CLN2,Opt,NaN,110.84,+0.56 (+0.51%),110.28,110.56,111.20,109.51,17492,23:09:27 CT 22 May 2022
1,AUG 2022CLQ2,Opt,NaN,107.75,+0.58 (+0.54%),107.17,107.34,108.03,106.47,2940,23:09:14 CT 22 May 2022
2,SEP 2022CLU2,Opt,NaN,104.55,+0.61 (+0.59%),103.94,104.11,104.75,103.39,1122,23:09:14 CT 22 May 2022
3,OCT 2022CLV2,Opt,NaN,101.69,+0.62 (+0.61%),101.07,100.85,101.78,100.56,184,22:44:28 CT 22 May 2022
4,NOV 2022CLX2,Opt,NaN,99.39,+0.69 (+0.70%),98.70,98.31,99.39,98.09,168,22:14:21 CT 22 May 2022
...,...,...,...,...,...,...,...,...,...,...,...
123,OCT 2032CLV32,Opt,NaN,-,-,64.58,-,-,-,0,17:02:03 CT 22 May 2022
124,NOV 2032CLX32,Opt,NaN,-,-,64.67,-,-,-,0,17:02:03 CT 22 May 2022
125,DEC 2032CLZ32,Opt,NaN,-,-,64.62,-,-,-,0,17:02:03 CT 22 May 2022


In [97]:
# Drop the duplicated headers

wti_prices_df = wti_prices_df.droplevel(0,axis=1)
wti_prices_df

,Month,Options,Chart,Last,Change,PriorSettle,Open,High,Low,Volume,Updated
0,JUL 2022CLN2,Opt,NaN,110.84,+0.56 (+0.51%),110.28,110.56,111.20,109.51,17492,23:09:27 CT 22 May 2022
1,AUG 2022CLQ2,Opt,NaN,107.75,+0.58 (+0.54%),107.17,107.34,108.03,106.47,2940,23:09:14 CT 22 May 2022
2,SEP 2022CLU2,Opt,NaN,104.55,+0.61 (+0.59%),103.94,104.11,104.75,103.39,1122,23:09:14 CT 22 May 2022
3,OCT 2022CLV2,Opt,NaN,101.69,+0.62 (+0.61%),101.07,100.85,101.78,100.56,184,22:44:28 CT 22 May 2022
4,NOV 2022CLX2,Opt,NaN,99.39,+0.69 (+0.70%),98.70,98.31,99.39,98.09,168,22:14:21 CT 22 May 2022
...,...,...,...,...,...,...,...,...,...,...,...
123,OCT 2032CLV32,Opt,NaN,-,-,64.58,-,-,-,0,17:02:03 CT 22 May 2022
124,NOV 2032CLX32,Opt,NaN,-,-,64.67,-,-,-,0,17:02:03 CT 22 May 2022
125,DEC 2032CLZ32,Opt,NaN,-,-,64.62,-,-,-,0,17:02:03 CT 22 May 2022
126,JAN 2033CLF33,Opt,NaN,-,-,64.74,-,-,-,0,17:02:03 CT 22 May 2022


In [98]:
if datetime.today().date().day > 15:
    time1 =datetime.today() + pd.Timedelta(days=31)
    print([time1.date().strftime("%b %Y"),np.nan,np.nan,np.nan,np.nan, wti_prices_df.iloc[0]['Last'],np.nan, np.nan, np.nan, np.nan, np.nan])

['Jun 2022', nan, nan, nan, nan, '110.84', nan, nan, nan, nan, nan]


In [99]:
# Insert a row with the with the most recent data to be current price

if datetime.today().date().day > 15:
    time1 =datetime.today() + pd.Timedelta(days=31)
    wti_prices_df.iloc[-1] = [time1.date().strftime("%b %Y"),np.nan,np.nan,np.nan,np.nan, wti_prices_df.iloc[0]['Last'],np.nan, np.nan, np.nan, np.nan, np.nan]
else:
    wti_prices_df.iloc[-1] = [datetime.today().date().strftime("%b %Y"),np.nan,np.nan,np.nan,np.nan, wti_prices_df.iloc[0]['Last'],np.nan, np.nan, np.nan, np.nan, np.nan]

In [100]:
# Convert Month column to datetime object
wti_prices_df['Month'] = pd.to_datetime(wti_prices_df['Month'].str.slice(0,8))

In [101]:
# Reduce to a two column DataFrame that contains Month as the index and PriorSettle as the price column
reduced_wti = wti_prices_df[["Month","PriorSettle"]]


In [102]:

# Rename columns
oil_df = reduced_wti.rename(columns={"Month":"Date","PriorSettle":"$/BBL"})


In [103]:

# Set Date as Index
oil_df = oil_df.set_index(["Date"]).sort_index()
oil_df.head()

,$/BBL
Date,
2022-06-01,110.84
2022-07-01,110.28
2022-08-01,107.17
2022-09-01,103.94
2022-10-01,101.07


In [104]:
# Split out the current year in order to have monthly prices through end of year
# Current year from datetime
current_year = datetime.now().year
current_year

2022

In [105]:
# Slice dataframe using index == current year
current_year_oil = oil_df.loc[oil_df.index.year == current_year] 

In [106]:
# Roll up the months of the remaining years in order to get a yearly price average
# rule='AS' to get dates to be Year Start ('A' is Year End)
out_years_oil = oil_df.loc[oil_df.index.year != current_year].astype(float).resample(rule='AS').mean().round(2)

In [107]:
strip_oil_price = current_year_oil.append(out_years_oil)
strip_oil_price

,$/BBL
Date,
2022-06-01,110.84
2022-07-01,110.28
2022-08-01,107.17
2022-09-01,103.94
2022-10-01,101.07
2022-11-01,98.7
2022-12-01,96.7
2023-01-01,88.62
2024-01-01,79.57


In [108]:
# Append the strip oil price to the historical oil price to get the complete dataset
complete_oil_price_df = monthly_historical_prices.append(strip_oil_price)
complete_oil_price_df

,$/BBL
Date,
1986-01-01,22.93
1986-02-01,15.45
1986-03-01,12.61
1986-04-01,12.84
1986-05-01,15.38
...,...
2029-01-01,64.52
2030-01-01,64.22
2031-01-01,64.42


In [109]:
complete_oil_price_df.tail(25)

,$/BBL
Date,
2021-11-01,79.15
2021-12-01,71.71
2022-01-01,83.22
2022-02-01,91.64
2022-03-01,108.5
2022-04-01,101.78
2022-05-01,106.59
2022-06-01,110.84
2022-07-01,110.28


In [110]:
complete_oil_price_df.to_csv("oil_prices.csv")